In [1]:
from selenium import webdriver

In [10]:
#Path to Chrome Driver for windows
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)

#Open site
driver.get("https://stockx.com/sneakers")

#Click on the annoying pop-up
driver.implicitly_wait(10)
PopUpQuitButton = driver.find_element_by_xpath("//img[@src='//stockx-assets.imgix.net/AllInAsk/close (1).svg?auto=compress,format']").click()

In [12]:
#For loop to loop through all 40 pairs on the first page
for i in range(1):
    
    #Click on the i + 1 sneaker
    xpathstr = "//div[@class='tile browse-tile updated']["+str(i+1)+"]"
    SneakerSelect = driver.find_element_by_xpath(xpathstr).click()

    #Click on button to view all sales of the sneaker
    driver.find_element_by_xpath("//a[@data-testid='product-viewallsales']").click()

    #Loop to press load more button to load full table. It will press load more button 10 times and then it will break out of loop. If there is not up to 10 button presses, it will automatically leave the loop. This should allow a little over 100 of the sale history.
    press_count = 0
    while True:
        try:
            load_more_button = driver.find_element_by_xpath("//button[@class='button button-block button-white']").click()
            press_count += 1
            if press_count == 10:
                break
        except:
            break
    
    table = driver.find_element_by_xpath("//table[@id='480']").text

    #Return to initial page and refresh
    driver.back()
    driver.refresh()